<a href="https://colab.research.google.com/github/ict3104-p2-team08/ict3104-p2-team08/blob/inference_feat/3104_T08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity Detection ML Test

## 1. Prerequisites

This section provides the setting up of the environment

In [ ]:
!pip install torch -f https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchaudio -f https://download.pytorch.org/whl/cu113/torchaudio-0.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchsummary==1.5.1 torchtext==0.13.1 torchvision -f https://download.pytorch.org/whl/cu113/torchvision-0.13.1%2Bcu113-cp37-cp37m-linux_x86_64.whl

In [ ]:
!git clone https://github.com/v-iashin/video_features.git
!pip install omegaconf==2.0.6

In [ ]:
# Imports widgets for jupyter
try:
    import piplite
    piplite.install(['ipywidgets'])
except ImportError:
    pass

# 2. Toggling of pipelines

This section shows the toggling of dependencies between TSU and STEP

##### Check if there is cuda is installed in your notebook

In [ ]:
#insert code here
import torch
print(torch.cuda.is_available())

## 3. Data exploration

This section load and display video data from the Toyota Smarthome (TSU) project

##### Jupyter -- Upload any files/datasets into the main folder

In [ ]:
#create everytime session start
!mkdir data
!mkdir data/input_files

In [ ]:
#cd to data -> input files folder to store uploaded vid
#%cd data/input_files/

#%cd ../../

In [4]:
import ipywidgets as widgets


In [ ]:
#file upload input
#from google.colab import files
#uploaded = files.upload()test = uploader.value
for name, file_info in uploader.value.items():
     with open (name, 'wb') as file:
          file.write(file_info['content'])

###### Google Colab

In [ ]:
#cd to data -> input files folder to store uploaded vid
#%cd data/input_files/

#%cd ../../

#file upload input
#from google.colab import files
#uploaded = files.upload()

In [ ]:
import ipywidgets as widgets
from ipyvuetify.extra import FileInput
import hashlib
file_input = FileInput()
file_input
# first select some files in the FileInput above

myfiles = file_input.get_files()
myfiles[0]
file_input.disabled = True

def md5_sum_file(file):
    file['file_obj'].seek(0)
    data = file['file_obj'].read()
    md5_sum = hashlib.md5(data).hexdigest()
    return f'{file["name"]}, md5: {md5_sum}'

sums = [md5_sum_file(file) for file in myfiles]
file_input.disabled = False

sums

In [5]:
uploader = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
    )


display(uploader)

FileUpload(value={}, description='Upload', multiple=True)

In [7]:
test = uploader.value
for name, file_info in uploader.value.items():
     with open (name, 'wb') as file:
          file.write(file_info['content'])

In [ ]:
import ipywidgets as widgets
from ipyvuetify.extra import FileInput
import hashlib
file_input = FileInput()
file_input
# first select some files in the FileInput above

myfiles = file_input.get_files()
myfiles[0]
file_input.disabled = True

def md5_sum_file(file):
    file['file_obj'].seek(0)
    data = file['file_obj'].read()
    md5_sum = hashlib.md5(data).hexdigest()
    return f'{file["name"]}, md5: {md5_sum}'

sums = [md5_sum_file(file) for file in myfiles]
file_input.disabled = False

sums

In [ ]:
#tsu video files to display in dropdown
from ipywidgets import  Dropdown, widgets
import os

FOLDER_PATH = 'input_files'
ROOT_PATH = './data'

file_list = os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH))


w = widgets.Dropdown(
    options= file_list,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

w.observe(on_change)

display(w)

In [ ]:
#display video - donote: only can within 30s vid
from IPython.display import HTML
from base64 import b64encode

video_path = './data/input_files/%' + w.value

locate= "" 

for i in video_path: 
   if i=='%': 
       pass
   else: 
       locate+=i 

print(locate)

def show_video(locate, video_width = 500):
  video_file = open(locate, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
  
show_video(locate)

In [ ]:
#get out of data/input_file dir to content dir
%cd ../
%cd ../

In [ ]:
### cd to video_features dir git clone
%cd video_features
%ls video_features


In [ ]:
### extract to I3D feature file
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import b
uild_cfg_path
from omegaconf import OmegaConf
import torch

# Select the feature type
feature_type = 'i3d'
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = [locate]
args.flow_type = 'raft'

extractor = ExtractI3D(args)

# Extract features
for video_path in args.video_paths:
    print(f'Extracting for {video_path}')
    feature_dict = extractor.extract(video_path)
    [(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]

In [ ]:
#derive rgb, flow, rgb+flow dict
rgb = list(feature_dict.items())[0]
flow = list(feature_dict.items())[1]

rgb_dict = {rgb[0]: rgb[1]}
flow_dict = {flow[0]: flow[1]}
rgb_flow_dict = {rgb[0]: rgb[1], flow[0]: flow[1]}

In [ ]:
#get out of video_features dir to content dir
%cd ../

In [ ]:
#cd to data -> input_files dir
%cd data/input_files

In [ ]:
#save dict as .npy feature file at the dir
import numpy as np

#rgb .npy
np.save(w.value[:-4] + "_rgb.npy", rgb_dict)

#flow .npy
np.save(w.value[:-4] + "_flow.npy", flow_dict)

#rgb_flow .npy
np.save(w.value[:-4] + "_rgb_flow.npy", rgb_flow_dict)

In [ ]:
#check if .npy load 
rbg_test =np.load("P02T03C03_rgb.npy", allow_pickle= True)
flow_test =np.load("P02T03C03_flow.npy", allow_pickle= True)
rbg_flow_test =np.load("P02T03C03_rgb_flow.npy", allow_pickle= True)


print(rbg_test)
print(flow_test)
print(rbg_flow_test)

## 4. Inference with pretrained model

This section allows generation of caption videos

### Select a pre-trained model

In [ ]:
%cd C:\Users\Tiffany\Desktop\SIT\S3T1\3104\3104 Project\ict3104-p2-team08

In [ ]:
#tsu video files to display in dropdown
import ipywidgets as widgets
import os

MODEL_PATH = 'models'
MODEL_ROOT_PATH = './Toyota_Smarthome/pipline/'

file_list = os.listdir(os.path.join(MODEL_ROOT_PATH, MODEL_PATH))
print(os.path.join(MODEL_ROOT_PATH, MODEL_PATH))

model_dropdown = widgets.Dropdown(
    options=file_list,
    description='Model:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

model_dropdown.observe(on_change)
display(model_dropdown)

### Choose an input video

In [ ]:
import json
VIDEO_PATH = 'input_files'
VIDEO_ROOT_PATH = './data/'

full_testable_video = os.listdir(os.path.join(VIDEO_ROOT_PATH, VIDEO_PATH))

model_name = model_dropdown.value
JSON_FILEPATH = "./Toyota_Smarthome/pipline/model_videos.json"
f = open(JSON_FILEPATH)
data = json.load(f)
testable_video = data[model_name]
testable_video = [video + ".mp4" for video in testable_video]
print(testable_video)

if model_name == "PDAN":
    file_list = full_testable_video
else:
    file_list = testable_video

video_dropdown = widgets.Dropdown(
    options=file_list,
    description='Video:',
    disabled=False,
)


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

video_dropdown.observe(on_change)
display(video_dropdown)

### Generate video with caption

In [ ]:
%pip install opencv-python

In [ ]:
#insert codes here
from IPython.display import Video

video_selected = video_dropdown.value.replace('.mp4','')
model_selected = model_dropdown.value
model_path = "./Toyota_Smarthome/pipline/models/" + model_selected
%run -i ./Toyota_Smarthome/pipline/test2.py -videofile={video_selected} -load_model={model_path} -name={model_selected}
Video("./Toyota_Smarthome/pipline/video_output/" + video_selected + "_caption.mp4")

## 5. Testing and evaluating trained models

This section shows

## 6. Training of new model

This section shows

### Choose datasets for training/testing

In [ ]:
#C:\Users\leech\OneDrive\Documents\untrimmed\RGB.tar\Videos_mp4
import os
import ipywidgets as widgets
DATASET_PATH = 'Videos_mp4'
DATASET_ROOT_PATH = '../OneDrive/Documents/untrimmed/RGB.tar/'

file_list = os.listdir(os.path.join(DATASET_ROOT_PATH, DATASET_PATH))

sel = widgets.SelectMultiple(
options=file_list,
#rows=10,
description='Dataset:',
disabled=False
)

selected_videos = []

but = widgets.Button(description = 'Upload')
def on_button_clicked(b):
    print(sel.value)

but.on_click(on_button_clicked)

display(sel)
display(but)

### Model name, Batch size, Epochs

In [ ]:
inp = widgets.Text(description='Model name:')
batch_size = widgets.BoundedIntText(description='Batch size:',
                                   value=2,
                                   max=4)
epochs = widgets.BoundedIntText(description='Epochs:',
                               value=1000,
                               max=1000)
button = widgets.Button(description="Enter")

button = widgets.Button(description="Enter")

def on_button_clicked(b):
    print("Model name set: ", inp.value)
    print("batch size set: ", batch_size.value)
    print("epochs set: ", epochs.value)

button.on_click(on_button_clicked)
display(inp)
display(batch_size)
display(epochs)
display(button)

### Start training model

In [ ]:
model_name = inp.value
selected_videos = list(sel.value)
for i in range(len(selected_videos)):
    selected_videos[i] = selected_videos[i].replace('.mp4','')
selected_videos_string=(','.join(selected_videos))
batchSize = str(batch_size.value)
epochValue = (epochs.value)
%run -i ./Toyota_Smarthome/pipline/train.py -video_train_test={selected_videos_string} -model=PDAN -name={model_name} -batch_size={batchSize} -epoch={epochValue}